# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome()

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [15]:
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [16]:
last_name = driver.find_element_by_name("pht_lnm")

In [17]:
last_name.send_keys("Nguyen")

In [18]:
select = Select(driver.find_element_by_name('pht_status'))
select.select_by_visible_text('Cosmetologists')

In [19]:
button = driver.find_element_by_xpath('//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]')
button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [20]:
entries = driver.find_elements_by_tag_name('tr')
for row in entries[:10]:
    print(row.text)

Name and Location Order Basis for Order
NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $1,250. Responde

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [21]:
for entry in entries[:10]:
    try: 
        name = entry.find_element_by_class_name ('results_text')
        print(name.text)
    except:
        pass
    

NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [22]:
for entry in entries[:10]:
    try: 
        violation = entry.find_elements_by_tag_name ('td')
        print(violation[2].text)
    except:
        pass

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [23]:
for cells in entries[:10]:
    try: 
        number = cells.find_elements_by_class_name ('results_text')
        print(number[-2].text)
    except:
        pass
    

COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [29]:
rows = driver.find_elements_by_tag_name('tr')
violations = []
for row in rows [:10]:
    try:
        name = row.find_element_by_class_name ('results_text')
        viol_desc = row.find_elements_by_tag_name ('td')
        viol_nmbr = row.find_elements_by_class_name ('results_text')
        lic_number = row.find_elements_by_class_name ('results_text')
        zip_cd = row.find_elements_by_class_name ('results_text')
        county = row.find_elements_by_class_name ('results_text')
        city = row.find_elements_by_class_name ('results_text')
        vltns_cosmetologists = {}
        vltns_cosmetologists['Name'] = name.text
        vltns_cosmetologists['Violation_Description'] = viol_desc[2].text
        vltns_cosmetologists['Violation_number'] = viol_nmbr[-2].text
        vltns_cosmetologists['License_Numbers'] = lic_number[-3].text
        vltns_cosmetologists['Zip_Code'] = zip_cd[-4].text
        vltns_cosmetologists['County'] = county[-5].text
        vltns_cosmetologists['City'] = city[-6].text
        violations.append(vltns_cosmetologists)
    except:
        pass
        


### Save that to a CSV

- Tip: You'll want to use pandas here

In [37]:
import pandas as pd
df = pd.DataFrame(violations)

In [32]:
df.to_csv("violations.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [36]:
pd.read_csv('/Users/nickospi/Desktop/violations.csv')

,City,County,License_Numbers,Name,Violation_Description,Violation_number,Zip_Code
0,SAN ANTONIO,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289,78217
1,EL PASO,EL PASO,737708,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594,79934
2,LONGVIEW,GREGG,731665,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257,75604
3,HOUSTON,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915,77014
4,SAN ANTONIO,BEXAR,767339,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",COS20180009255,78255
5,ARLINGTON,TARRANT,681274,"NGUYEN, TUAN A",Respondent failed to clean and disinfect all w...,COS20140018343,76011
6,EULESS,TARRANT,"721373, 1142884","NGUYEN, THAO B",Respondent failed to clean and sanitize whirlp...,COS20180008846,76039
7,HOUSTON,HARRIS,1470271,"NGUYEN, BETH MARIA",The Respondent's license was revoked upon Resp...,COS20180000897,77083
8,AMARILLO,POTTER,"1196244, 767015, 767014","NGUYEN, TRUNG N","Respondent failed to clean, disinfect, and ste...",COS20170023893,79106
